In [1]:
from keras.preprocessing import sequence
from keras.models import Sequential, Model, load_model
from keras.layers import GRU, Dense, Dropout, Input, concatenate
from keras.utils import np_utils
from keras.optimizers import Adam
from copy import deepcopy
import json
import pandas as pd
import numpy as np
import pickle
import math
import datetime
import tensorflow as tf


Using TensorFlow backend.


In [2]:
# variable
words = [
    'come quickly', 'emergency', 'father', 'fever', 'good luck',
    'headache', 'hello', 'help', 'hi', 'hungry',
    'like', 'mother', 'mother_father', 'mother_mother', 'not ok',
    'quickly', 'sorry', 'tomorrow', 'yogurt'
]
data_per_word = 27
# data_length = data_per_word * len(words)
speedup_max = 1.15
timesteps = 50
augment_timesteps = math.ceil(timesteps * speedup_max) # add speedup
dimensions = 34 #22 #34
pick_frame_every_no = 12

In [3]:
with open('./extracted_feature/original_full_32_dimensions_GridSearch.pkl', 'rb') as handle:
    [x_original, y_original, x_augment] = pickle.load(handle)

In [ ]:
# [
#     {
#         "hands": {
#             "right": {
#                 "arm": {
#                     "direction": [x,y,z]
#                 }
#                 "hand_palm_position": [x,y,z],
#                 "roll": n,
#                 "yaw": n,
#                 "pitch": n,
#                 "fingers": {
#                     "pinky": {
#                         "bones": {
#                             "distal": {
#                                 "next_joint": [x,y,z]
#                             },
#                             "proximal": {
#                                 "direction": [x,y,z]
#                             }
#                         }
#                     }
#                 }
#             }
#         }
#     }
# ]

In [4]:
def get_feature(frame):
    finger_name = ['thumb', 'index', 'middle', 'ring', 'pinky']
    hand_name = ['left', 'right']
    
    fingertip_pos = np.zeros([2, 5, 3])
    
    hand_palm_pos = np.zeros([2, 3])
    hand_palm_rpy = np.zeros([2, 3])
    hand_palm_unit_vector = np.zeros([2, 3])
    arm_direction = np.zeros([2, 3])
    
    feature = np.zeros([34])
    
    for hand_idx, hand in enumerate(hand_name):
        if not hand in frame['hands']:
            continue
        hand_palm_pos[hand_idx] = np.array(frame['hands'][hand]['hand_palm_position'])
        hand_palm_rpy[hand_idx] = np.array([ frame['hands'][hand]['roll'],
                                                    frame['hands'][hand]['pitch'],
                                                    frame['hands'][hand]['yaw']])
#         hand_palm_unit_vector[hand_idx] = np.cross(
#             frame['hands'][hand]['fingers']['index']['bones']['proximal']['direction'],
#             frame['hands'][hand]['fingers']['middle']['bones']['proximal']['direction']
#         )
#         arm_direction[hand_idx] = np.array(frame['hands'][hand]['arm']['direction'])
        for finger_idx, finger in enumerate(finger_name):
            fingertip_pos[hand_idx, finger_idx] = (
                np.array(frame['hands'][hand]['fingers'][finger]['bones']['distal']['next_joint'])
            )
            
    fingertip_pos_shift = np.roll(fingertip_pos, 1, axis=1)
    dist_btw_fingertip = np.linalg.norm(fingertip_pos - fingertip_pos_shift, axis=2)
        
    feature[0:6] = hand_palm_pos.reshape(6)
    feature[6:12] = hand_palm_rpy.reshape(6)
    feature[12:22] = dist_btw_fingertip.reshape(10)
#     feature[22:28] = hand_palm_unit_vector.reshape(6)
#     feature[28:34] = arm_direction.reshape(6)
    
    return feature
    

In [5]:
def get_timesteps(json_data, pick_frame_every_no): 
    timesteps = np.zeros([0, dimensions])
    for frame_no, frame in enumerate(json_data):
        if frame_no % pick_frame_every_no != 0 or not frame['hands']:
            continue
        feature = get_feature(frame)
        timesteps = np.vstack((timesteps, feature))
    return timesteps

In [6]:
def get_x_norm(x):
    x_std = x.std(axis=(0,1), keepdims=True)
    x_mean = x.mean(axis=(0,1), keepdims=True)
    x_norm = (x-x_mean)/x_std
    return x_norm

In [7]:
def get_x_augment_custom(x_augment = x_augment):
    x_augment_custom = {}
    for key, pack in x_augment.items():
        x_augment_custom[key] = pack[:,:,0:22]
    return x_augment_custom

In [8]:
file_name = 'model_grid_search_[0.85, 0.9, 1.0, 1.1, 1.15]_128_1_Note_try old features'

try:
    f = h5py.File('./interested_model/'+ file_name +'.h5', 'r+')
    del f['optimizer_weights']
    f.close()
except:
    None

model = load_model("./interested_model/"+ file_name +".h5")
# with open('./interested_model/history_'+ file_name +'.pkl', 'rb') as handle:
#     history = pickle.load(handle)

C:\Users\Mike\Anaconda3\lib\site-packages\keras\models.py:291: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [9]:
def get_x():
    timesteps = augment_timesteps
    x_original = np.zeros([0, timesteps, dimensions])
    word = 'emergency'
    data_per_word = 27
    
    for data_no in range(data_per_word):
        try:
            with open("./record/{}/json_{}.txt".format(word, data_no)) as json_data:
                json_data = json.load(json_data)
        except Exception as s:
            print ('error log:', s)

        _timesteps = get_timesteps(json_data, pick_frame_every_no)
        _timesteps = np.vstack((_timesteps, np.zeros([timesteps - _timesteps.shape[0], dimensions])))
        x_original = np.vstack((x_original, [_timesteps]))
        
    return x_original

In [10]:
def get_x2():
    data = []
    word = 'emergency'
    data_per_word = 1
    
    for data_no in range(data_per_word):
        try:
            with open("./record/{}/json_{}.txt".format(word, data_no)) as json_data:
                json_data = json.load(json_data)
        except Exception as s:
            print ('error log:', s)
        
        data.append(json_data)
        
    return sampling_x(data)

In [11]:
def sampling_x(data):
    timesteps = augment_timesteps
    x_original = np.zeros([0, timesteps, dimensions])
    
    for pack in data:
        _timesteps = get_timesteps(pack, pick_frame_every_no)
        _timesteps = np.vstack((_timesteps, np.zeros([timesteps - _timesteps.shape[0], dimensions])))
        x_original = np.vstack((x_original, [_timesteps]))
        
    return x_original
    

In [50]:
def predict(x_test):
    x_std = x_original.std(axis=(0,1), keepdims=True)
    x_mean = x_original.mean(axis=(0,1), keepdims=True)

#     print(x_test.shape)
    
    x_predict = x_test
    x_predict = (x_predict-x_mean)/x_std
    x_predict = x_predict[:,:,0:22]

#     print(x_predict.shape)
    prediction = model.predict(x_predict, verbose = 1)
#     print('prediction', prediction)
    
    result_text = []
    
    prediction_word_index = np.argmax(prediction, axis = 1)
    
    for ans in prediction_word_index:
        result_text.append(words[ans])
    
#     print('result_text', result_text)
    
    return result_text

In [1]:
def predict_with_data(data):
    x_predict = sampling_x(data)
    
    return predict(x_predict)

In [14]:
def test_predict():
    with open('./extracted_feature/original_test_set_2.pkl', 'rb') as handle:
        [x_3rd_user, y_3rd_user, x_3rd_user_augment] = pickle.load(handle)
    
    predict(x_3rd_user_augment[1.00][0:1])

In [15]:
def test_predict2():
    x_predict = get_x2()
    
    predict(x_predict)

In [44]:
def test_predict3(word = 'hi', index = 1):
    data = []
    data_per_word = index
    
    for data_no in range(data_per_word):
        try:
            with open("./record/{}/json_{}.txt".format(word, data_no)) as json_data:
                json_data = json.load(json_data)
        except Exception as s:
            print ('error log:', s)
        
        data.append(json_data)

    return predict_with_data(data)

In [51]:
print('res :', test_predict3())

1/1 [==============================] - 0s 4ms/step
res : ['hi']
